In [1]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library(patchwork)
library(forcats)
library(Polychrome)
library(grid)

Attaching SeuratObject

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
xenium.obj = qs::qread('230710_xenium_sct_unimapped_cca_polar_label.qs')
xenium.obj

An object of class Seurat 
502 features across 78054 samples within 6 assays 
Active assay: SCT (100 features, 100 variable features)
 5 other assays present: Xenium, BlankCodeword, ControlCodeword, ControlProbe, prediction.score.cell_class
 3 dimensional reductions calculated: umap, cca, ref.umap
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [4]:
xenium.obj %>% `[[` %>% colnames

[1] "orig.ident"                     "nCount_Xenium"                 
 [3] "nFeature_Xenium"                "nCount_BlankCodeword"          
 [5] "nFeature_BlankCodeword"         "nCount_ControlCodeword"        
 [7] "nFeature_ControlCodeword"       "nCount_ControlProbe"           
 [9] "nFeature_ControlProbe"          "cell_area"                     
[11] "density"                        "elongation"                    
[13] "avg_confidence"                 "nucleus_area"                  
[15] "sample_name"                    "treatment"                     
[17] "strain"                         "time"                          
[19] "nCount_SCT"                     "nFeature_SCT"                  
[21] "predicted.cell_class.score"     "predicted.cell_class"          
[23] "predicted.cell_class.score_2s"  "predicted.cell_class_2s"       
[25] "predicted.polar_label.score_2s" "predicted.polar_label_2s"

In [4]:
obj_fgf1 = qs::qread('obj_fgf1_obob5v5.qs')
obj_fgf1

An object of class Seurat 
58613 features across 58235 samples within 3 assays 
Active assay: SCT (26292 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)
p1 = DimPlot(xenium.obj, reduction = "umap", group.by = "predicted.labels", label = TRUE, repel = TRUE) + theme(legend.position = "none")  + labs(title = "Spatial on spatial UMAP")
p2 = DimPlot(xenium.obj, reduction = "ref.umap", group.by = "predicted.labels", label = TRUE, repel = TRUE) + theme(legend.position = "none") + labs(title = "Spatial on snRNAseq reference UMAP")
p3 = DimPlot(obj_fgf1, reduction = "umap", group.by = "labels", label = TRUE, repel = TRUE) + theme(legend.position = "none") + labs(title = "snRNAseq reference")
options(repr.plot.width=30, repr.plot.height=10)
p1 + p2 + p3



In [5]:
create_grouping_col <- function(seurat_obj, prefix) {
  new_metadata <- case_when(str_detect(seurat_obj@meta.data$predicted.labels, fixed(prefix)) ~ seurat_obj@meta.data$predicted.labels,
                            TRUE ~ NA_character_) %>% as.factor
  new_metadata = new_metadata %>% fct_relevel(sort(na.omit(levels(new_metadata))), after = Inf)
  return(new_metadata)
}



In [6]:
plot_celltype_idp = function(seurat_obj, cell_str, fov='fov'){
    new_grouping = seurat_obj %>%
        create_grouping_col(cell_str)
    seurat_obj = AddMetaData(seurat_obj, new_grouping, col.name = 'grouping_col')

    # Get the levels of the lab_Pomc column, including the NA_character_ level
    grouping_levels <- levels(seurat_obj@meta.data$grouping_col)
    # Generate colors using the polychrome palette
    how_many_colors = length(grouping_levels)
    colors <- Polychrome::sky.colors(how_many_colors) %>% as.character %>% `[`(1:how_many_colors) # Excluding the NA_character_ level
    # Add 'gray10' for the NA_character_ level
    colors <- c(colors, "gray10")
    # Create a named vector of colors, with levels as names
    named_colors <- setNames(colors, grouping_levels)
    # Map the lab_Pomc column values to the corresponding colors
    # color_vector <- mapvalues(xenium.obj@meta.data$lab_Pomc, from = lab_Pomc_levels, to = named_colors)
    
    ggp = ImageDimPlot(seurat_obj,
                       group.by='grouping_col',
                       boundaries = 'segmentation',
                       fov = fov,
                       border.size =0.1, 
                       border.color =NA,
                       na.value='gray20', 
                       cols=named_colors, 
                       axes = FALSE,
                       size=0)
    ggp
}

In [7]:
make_idp_pdf = function(seurat_obj, cell_type, pdf_folder=""){
    a1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov')  + theme(legend.position = "none")
    a2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.1') + theme(legend.position = "none")
    b1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.2') + theme(legend.position = "none")
    b2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.3') + theme(legend.position = "none")
    c1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.4') + theme(legend.position = "none")
    c2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.5') + theme(legend.position = "none")
    d1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.6') + theme(legend.position = "none")
    d2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.7') + theme(legend.position = "none")
    title_theme <- theme(plot.title = element_text(color = "white", size = 8, hjust = 0.5))
    # Set black background theme for each plot
    legend_theme <- theme(legend.position = "bottom",
                          legend.direction = "horizontal",
                          legend.box = "horizontal",
                          legend.text = element_text(size = 4),
                          legend.key.size = unit(2, "mm"))
    # black_background_theme <- theme(plot.background = element_rect(fill = "black"),
    #                                 panel.background = element_rect(fill = "black"))
    black_background_theme <- theme(plot.background = element_rect(fill = "black", color = 'black'),
                          panel.background = element_rect(fill = "black", color = 'black'),
                          plot.margin = margin(0, 0, 0, 0),
                          panel.grid.major = element_blank(),
                          panel.grid.minor = element_blank(),
                          panel.border = element_blank())

    a1_with_title <- a1 + theme(legend.position = "right") + ggtitle("a1_Veh_PF") + title_theme + legend_theme + black_background_theme
    a2_no_legend_with_title <- a2 + guides(scale="none") + ggtitle("a2_Veh_PF") + title_theme + black_background_theme
    b1_no_legend_with_title <- b1 + guides(scale="none") + ggtitle("b1_Veh_PF") + title_theme + black_background_theme
    b2_no_legend_with_title <- b2 + guides(scale="none") + ggtitle("b2_Veh_PF") + title_theme + black_background_theme
    c1_no_legend_with_title <- c1 + guides(scale="none") + ggtitle("c1_FGF1") + title_theme + black_background_theme
    c2_no_legend_with_title <- c2 + guides(scale="none") + ggtitle("c2_FGF1") + title_theme + black_background_theme
    d1_no_legend_with_title <- d1 + guides(scale="none") + ggtitle("d1_FGF1") + title_theme + black_background_theme
    d2_no_legend_with_title <- d2 + guides(scale="none") + ggtitle("d2_FGF1") + title_theme + black_background_theme

    # Combine the plots using patchwork with a black background
    combined_plots <- a1_with_title + c1_no_legend_with_title +
                      a2_no_legend_with_title + c2_no_legend_with_title +
                      b1_no_legend_with_title + d1_no_legend_with_title +
                      b2_no_legend_with_title + d2_no_legend_with_title +
                      plot_layout(guides = "collect", ncol = 2, heights = c(1, 1, 1, 1), widths = c(1, 1))

    # combined_plots = combined_plots & black_background_theme & legend_theme 
    combined_plots = combined_plots + plot_annotation(title=cell_type, theme=black_background_theme) + plot_annotation(theme=legend_theme) + plot_annotation(theme=title_theme)
    # Save the combined_plots to a PDF with a black background
    pdf_name = paste0(pdf_folder, cell_type, '_ct_mapping.pdf')
    pdf(pdf_name, width = 8.27, height = 11.69, bg = "black")
    # grid.newpage()
    grid.draw(combined_plots)
    dev.off()
}

In [8]:
cell_types = xenium.obj %>% `[[` %>% group_by(predicted.labels) %>% summarise(n = n()) %>% arrange(desc(n)) %>% pull(predicted.labels) %>% unique
cell_types

[1] "Nr5a1_Bdnf"                "Unassigned2"              
 [3] "a1_Tany-Astrocytes"        "Agrp"                     
 [5] "Astrocytes"                "MOL"                      
 [7] "Slc17a6_Fam19a2"           "NG2_OPC2"                 
 [9] "Htr3b"                     "Rgs16_Nmu"                
[11] "Pomc_Lepr"                 "Pomc_Glipr1-Slc17a6_Trhr" 
[13] "Trh_Cxcl12"                "Tmem215"                  
[15] "Nr5a1_Nfib"                "Nr5a1_Bdnf-Unassigned1"   
[17] "Trh_Lef1"                  "Ghrh"                     
[19] "b1_Tanycytes"              "Kisspeptin"               
[21] "Unassigned1.sc17.sc28"     "Sst_Pthlh.sc21"           
[23] "Qrfp"                      "Epend"                    
[25] "Sst_Pthlh.sc23"            "Slc17a6_Trhr"             
[27] "Unassigned1.sc39"          "Rgs16_Dlx1-Unassigned2"   
[29] "Sst_Unc13c-Agrp"           "a2_Tanycytes"             
[31] "Fibroblasts2-Fibroblasts3" "Nfix_Htr2c"               
[33] "Rgs16_Dlx1"                "Th_Nfib-Arx_Nr5a2"        
[35] "Tbx19"                     "Nr5a1_Bdnf-Nr5a1_Nfib"    
[37] "b2_Tany"                   "Gm8773_Tac1-Hdc"          
[39] "Th_Slc6a3"                 "Gpr50"                    
[41] "NFOL"                      "Mural_Cells2"             
[43] "Sst_Nts"                   "a2_Tanycytes-b1_Tanycytes"
[45] "Microglia"                 "Unassigned1.sc37"         
[47] "Sst_Pthlh-Ghrh"            "Parstuber2A-ParsTuber1"   
[49] "Oxt-Unassigned1"           "Unassigned2-Unassigned1"  
[51] "MOL-a2_Tanycytes"          "Endothelial"              
[53] "NG2_OPC2-Microglia"

In [ ]:
for (cell_type in cell_types){
    print(cell_type)
    make_idp_pdf(xenium.obj, cell_type, pdf_folder="ct_mappings_cca_fgf1XEgenesUMAP/")
}

[1] "Nr5a1_Bdnf"
[1] "Unassigned2"
[1] "a1_Tany-Astrocytes"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”


[1] "Agrp"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”


[1] "Astrocytes"


...

Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”


[1] "Htr3b"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”


[1] "Rgs16_Nmu"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
